In [7]:
import plotly.express as px
import dash
from dash import dcc, html, dash_table, Dash
from dash.dependencies import Input, Output
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.stats import norm

In [10]:
def haversine(row):
    lat1, lon1, lat2, lon2 = map(np.radians,[row.lat_s, row.lng_s, row.lat_d, row.lng_d])
    dlat, dlon = lat2-lat1, lon2-lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return 6371 * 2 * np.arcsin(np.sqrt(a))

ship_ss = pd.read_csv('data/ship_sub2store.csv')
nodes = pd.read_csv('data/nodes.csv')[['node_id','lat','lng']]
stores = pd.read_csv('data/stores.csv')[['store_name','region']]
stores_meta = stores.rename(columns={'store_name':'dest'})
svc_req = pd.read_csv('data/service_requirements.csv')[
    ['region','speed_kmph','service_window','fill_rate']
]
rates = pd.read_csv('data/rates.csv')[['mode','rate_per_km']]
df = ship_ss.merge(
    nodes.rename(columns={'node_id':'source','lat':'lat_s','lng':'lng_s'}),
    on='source', how='left'
).merge(
    nodes.rename(columns={'node_id':'dest','lat':'lat_d','lng':'lng_d'}),
    on='dest', how='left'
).merge(
    stores_meta, on='dest', how='left'
).merge(
    svc_req, on='region', how='left'
)

df['distance_km']     = df.apply(haversine, axis=1)
df['service_window_hrs'] = df['service_window'].str.extract(r'(\d+)').astype(float)
df['lead_time_hrs']      = df['distance_km'] / df['speed_kmph']
df['delay_flag']         = df['lead_time_hrs'] > df['service_window_hrs']

ltl_rate = rates.loc[rates['mode']=='LTL','rate_per_km'].iloc[0]
df['estimated_cost']    = df['qty'] * df['distance_km'] * ltl_rate
threshold = df['estimated_cost'].mean() + df['estimated_cost'].std()
df['cost_overrun_flag'] = df['estimated_cost'] > threshold
df['route'] = df['source'] + '→' + df['dest']
agg = df.groupby('route', as_index=False).agg(
    total_qty         = ('qty','sum'),
    delay_cnt         = ('delay_flag','sum'),
    delay_rate        = ('delay_flag','mean'),
    cost_overrun_cnt  = ('cost_overrun_flag','sum'),
    avg_cost          = ('estimated_cost','mean')
)
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("风险预警看板", style={'textAlign':'center'}),
    dash_table.DataTable(
        id='risk-table',
        columns=[
            {'name':'路线','id':'route'},
            {'name':'发运量','id':'total_qty',        'type':'numeric','format':{'specifier':','}},
            {'name':'延误次数','id':'delay_cnt',      'type':'numeric'},
            {'name':'延误率','id':'delay_rate',       'type':'numeric','format':{'specifier':'.1%'}},
            {'name':'超支次数','id':'cost_overrun_cnt','type':'numeric'},
            {'name':'平均成本','id':'avg_cost',        'type':'numeric','format':{'specifier':',.0f'}}
        ],
        data=agg.to_dict('records'),
        style_cell={'padding':'5px','textAlign':'center'},
        style_header={'backgroundColor':'#f1f1f1','fontWeight':'bold'},
        style_data_conditional=[
            {
                'if': {'filter_query':'{delay_rate} > 0.2'},
                'backgroundColor':'#FFE5E5'
            },
            {
                'if': {'filter_query':'{cost_overrun_cnt} > 0'},
                'backgroundColor':'#FFF5E5'
            }
        ],
        page_size=10,
    ),

    html.Br(),
    dcc.Graph(id='delay-chart'),
    dcc.Graph(id='cost-chart'),
])

@app.callback(
    Output('delay-chart','figure'),
    Output('cost-chart','figure'),
    Input('risk-table','data')
)
def update_charts(rows):
    dff = pd.DataFrame(rows)

    fig_delay = px.bar(
        dff.sort_values('delay_cnt',ascending=False).head(10),
        x='route', y='delay_cnt',
        title="Top10 延误次数路由",
        labels={'delay_cnt':'延误次数','route':'路线'}
    )
    fig_delay.update_layout(margin={'t':40,'b':80,'l':40,'r':20})

    fig_cost = px.bar(
        dff.sort_values('cost_overrun_cnt',ascending=False).head(10),
        x='route', y='cost_overrun_cnt',
        title="Top10 成本超支路由",
        labels={'cost_overrun_cnt':'超支次数','route':'路线'}
    )
    fig_cost.update_layout(margin={'t':40,'b':80,'l':40,'r':20})

    return fig_delay, fig_cost

if __name__ == '__main__':
    app.run(port=8052, debug=True)

In [9]:
# 大前提：1月初的库存为0

policy      = pd.read_csv('data/inventory_policy.csv')
h_rate      = policy.loc[0, 'holding_cost_pct'] / 100    # 年持有率
unit_cost   = policy.loc[0, 'unit_value']                # 单位价值
K_order     = policy.loc[0, 'fixed_cost']                # 每次订货固定成本

warehouses  = pd.read_csv('data/warehouses.csv')['warehouse_name'].tolist()
# 2. 计算每个分仓的当期“实际库存” (年初 = 0)
# —— 入库量 = tot2sub.dest + sub2sub.dest
in1 = pd.read_csv('data/ship_tot2sub.csv')[['dest','qty']].rename(columns={'dest':'node','qty':'in'})
in2 = pd.read_csv('data/ship_sub2sub.csv')[['dest','qty']].rename(columns={'dest':'node','qty':'in'})
inbound = pd.concat([in1,in2],ignore_index=True).groupby('node',as_index=False)['in'].sum()

# —— 出库量 = sub2sub.source + sub2store.source
out1 = pd.read_csv('data/ship_sub2sub.csv')[['source','qty']].rename(columns={'source':'node','qty':'out'})
out2 = pd.read_csv('data/ship_sub2store.csv')[['source','qty']].rename(columns={'source':'node','qty':'out'})
outbound= pd.concat([out1,out2],ignore_index=True).groupby('node',as_index=False)['out'].sum()

inv_df = (pd.DataFrame({'node':warehouses[1:]})  # 只展示分仓
    .merge(inbound,  on='node', how='left')
    .merge(outbound, on='node', how='left')
    .fillna(0)
)
inv_df['actual_inv'] = inv_df['in'] - inv_df['out']


# 3. 准备月度需求统计（DC→下游）
# 合并 sub2sub/sub2store，两者都是分仓出货给下级
ship_sub2sub   = pd.read_csv('data/ship_sub2sub.csv')[['source','month','qty']]
ship_sub2store = pd.read_csv('data/ship_sub2store.csv')[['source','month','qty']]
ship_out       = pd.concat([ship_sub2sub, ship_sub2store], ignore_index=True)

# 统计每个分仓每月出货总量
month_stats = (ship_out
    .groupby(['source','month'], as_index=False)['qty']
    .sum()
    .rename(columns={'source':'node'})
)
# 再统计：月均、月波动
agg_month = (month_stats
    .groupby('node')['qty']
    .agg(mean_m='mean', std_m='std')
    .reset_index()
)

# 转换到日均需求及日波动
DAYS_PER_MONTH = 30
agg_month['D_daily']     = agg_month['mean_m'] / DAYS_PER_MONTH
agg_month['sigma_daily'] = agg_month['std_m']  / DAYS_PER_MONTH


# 4. 计算提前期 L（天），取自“总仓→分仓”平均 lead-time
# 4.1 读取 tot2sub 发运并算距离 & 时长
ship_t2s = pd.read_csv('data/ship_tot2sub.csv')[['source','dest','qty']]
# 合并坐标
t2s = (ship_t2s
    .merge(nodes.rename(columns={'node_id':'source','lat':'lat_s','lng':'lng_s'}), on='source')
    .merge(nodes.rename(columns={'node_id':'dest',  'lat':'lat_d','lng':'lng_d'}),   on='dest')
)
# haversine 函数请沿用之前定义
t2s['distance_km']    = t2s.apply(haversine, axis=1)
# 假设总仓到分仓的平均速度用 policy 中 throughput_max（只是举例），或者自行指定
# 这里直接用 policy 中的 throughput_max / 24h 来反推 avg speed（示例）
avg_speed_kmph = policy.loc[0,'throughput_max'] / 24
t2s['lead_time_days'] = t2s['distance_km'] / avg_speed_kmph

# 每个 DC 的平均 L
L_days = t2s.groupby('dest', as_index=False)['lead_time_days'].mean().rename(columns={'dest':'node','lead_time_days':'L'})


# 5. 安全库存 SS & 补货点 R & 经济订货量 Q
from scipy.stats import norm
Z98 = norm.ppf(0.98)

df = (agg_month
    .merge(L_days,     on='node', how='left')
    .merge(inv_df[['node','actual_inv']], on='node')
)

df['SS'] = Z98 * df['sigma_daily'] * np.sqrt(df['L'])
df['R']  = df['D_daily']*df['L'] + df['SS']

# 年度需求
df['D_annual'] = df['mean_m'] * 12
# 年持有成本 = h_rate * unit_cost
h = h_rate * unit_cost
df['EOQ'] = np.sqrt(2 * df['D_annual'] * K_order / h)


# 6. 最终结果表
result = df[[
    'node','actual_inv','SS','R','EOQ'
]].round({
    'actual_inv':0,'SS':0,'R':0,'EOQ':0
})


# 1) 一次性定义 app
app = Dash(__name__)

# 2) 一次性定义 layout
app.layout = html.Div([
    html.H2("分仓库存管理一览"),
    dash_table.DataTable(
        id='inv-mgmt',
        columns=[
            {'name':'节点','id':'node'},
            {'name':'当前库存','id':'actual_inv','type':'numeric','format':{'specifier':',.0f'}},
            {'name':'安全库存 SS','id':'SS','type':'numeric','format':{'specifier':',.0f'}},
            {'name':'补货点 R','id':'R','type':'numeric','format':{'specifier':',.0f'}},
            {'name':'建议订货量 Q','id':'EOQ','type':'numeric','format':{'specifier':',.0f'}},
        ],
        data=result.to_dict('records'),
        style_cell={'textAlign':'center','padding':'5px'},
        style_header={'backgroundColor':'#f9f9f9','fontWeight':'bold'},
        page_size=10,
    )
])

app.run(mode='inline',port=8055, debug=True)